In [ ]:
# Shopify Scripts Notebook

# This notebook is for generating csv tranformation exports to be imported into Shopify


In [1]:
# Install required packages (run this first)
%pip install pandas


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Create outputs directory
import os
os.makedirs('outputs', exist_ok=True)
print("Created outputs directory")

Created outputs directory


In [7]:
import pandas as pd
import re

def transform_wc_to_shopify(input_file, output_file):
    """
    Transform WooCommerce export CSV to Shopify import format
    """
    # Read the WooCommerce export
    df = pd.read_csv(input_file)
    
    # Create empty Shopify DataFrame with required columns
    shopify_columns = [
        'Handle', 'Title', 'Body (HTML)', 'Vendor', 'Product Category', 'Type', 'Tags', 'Published',
        'Option1 Name', 'Option1 Value', 'Option2 Name', 'Option2 Value', 'Option3 Name', 'Option3 Value',
        'Variant SKU', 'Variant Grams', 'Variant Inventory Tracker', 'Variant Inventory Qty', 
        'Variant Inventory Policy', 'Variant Fulfillment Service', 'Variant Price', 'Variant Compare At Price',
        'Variant Requires Shipping', 'Variant Taxable', 'Variant Barcode', 'Image Src', 'Image Position',
        'Image Alt Text', 'Gift Card', 'SEO Title', 'SEO Description', 'Google Shopping / Google Product Category',
        'Google Shopping / Gender', 'Google Shopping / Age Group', 'Google Shopping / MPN', 
        'Google Shopping / Condition', 'Google Shopping / Custom Product', 'Variant Image', 'Variant Weight Unit',
        'Variant Tax Code', 'Cost per item', 'Included / United States', 'Price / United States',
        'Compare At Price / United States', 'Included / International', 'Price / International', 
        'Compare At Price / International', 'Status'
    ]
    
    shopify_df = pd.DataFrame(columns=shopify_columns)
    
    for idx, row in df.iterrows():
        # Create handle from product name (URL-friendly)
        handle = re.sub(r'[^\w\s-]', '', str(row['Name'])).strip()
        handle = re.sub(r'[-\s]+', '-', handle).lower()
        
        # Extract images and create multiple rows if needed
        images = []
        if pd.notna(row['Images']) and row['Images']:
            images = [img.strip() for img in str(row['Images']).split(',')]
        
        # Convert weight from lbs to grams
        weight_grams = 0
        if pd.notna(row['Weight (lbs)']) and row['Weight (lbs)']:
            weight_grams = int(float(row['Weight (lbs)']) * 453.592)
        
        # Convert tags
        tags = ""
        if pd.notna(row['Tags']) and row['Tags']:
            tag_list = [tag.strip().replace('#', '') for tag in str(row['Tags']).split(',')]
            tags = ', '.join(tag_list)
        
        # Determine published status
        published = 'TRUE' if row['Published'] != -1 else 'FALSE'
        status = 'active' if published == 'TRUE' else 'archived'
        
        # Process description - remove all carriage returns, newlines, and line breaks
        description = ''
        if pd.notna(row['Description']):
            description = re.sub(r'[\r\n\x0b\x0c\x85\u2028\u2029]+', '', str(row['Description']))
        
        # Handle pricing logic
        # In WooCommerce: Sale price is the discounted price, Regular price is the original price
        # In Shopify: Variant Price is the selling price, Compare At Price is the original price
        variant_price = row['Sale price'] if pd.notna(row['Sale price']) and row['Sale price'] else row['Regular price']
        compare_at_price = row['Regular price'] if pd.notna(row['Sale price']) and row['Sale price'] and row['Sale price'] != row['Regular price'] else ''
        
        # Main product row
        new_row = {
            'Handle': handle,
            'Title': row['Name'],
            'Body (HTML)': description,
            'Vendor': '', # Don't touch this line
            'Product Category': row['Categories'] if pd.notna(row['Categories']) else '',
            'Type': '', # Don't touch this line
            'Tags': tags,
            'Published': published,
            'Option1 Name': '',
            'Option1 Value': '',
            'Option2 Name': '',
            'Option2 Value': '',
            'Option3 Name': '',
            'Option3 Value': '',
            'Variant SKU': row['SKU'],
            'Variant Grams': weight_grams,
            'Variant Inventory Tracker': 'shopify',
            'Variant Inventory Qty': row['Stock'] if pd.notna(row['Stock']) and row['In stock?'] else 0,
            'Variant Inventory Policy': 'deny',
            'Variant Fulfillment Service': 'manual',
            'Variant Price': variant_price if pd.notna(variant_price) else 0,
            'Variant Compare At Price': compare_at_price,
            'Variant Requires Shipping': 'TRUE',
            'Variant Taxable': 'TRUE' if row['Tax status'] == 'taxable' else 'FALSE',
            'Variant Barcode': '',
            'Image Src': images[0] if images else '',
            'Image Position': '1' if images else '',
            'Image Alt Text': row['Name'] if images else '',
            'Gift Card': 'FALSE',
            'SEO Title': row['Name'],
            'SEO Description': row['Short description'] if pd.notna(row['Short description']) else '',
            'Google Shopping / Google Product Category': '',
            'Google Shopping / Gender': '',
            'Google Shopping / Age Group': '',
            'Google Shopping / MPN': '', # Don't touch this line
            'Google Shopping / Condition': '',
            'Google Shopping / Custom Product': '',
            'Variant Image': '',
            'Variant Weight Unit': 'g',
            'Variant Tax Code': '',
            'Cost per item': '',
            'Included / United States': 'TRUE',
            'Price / United States': '',
            'Compare At Price / United States': '',
            'Included / International': 'TRUE',
            'Price / International': '',
            'Compare At Price / International': '',
            'Status': status
        }
        
        shopify_df = pd.concat([shopify_df, pd.DataFrame([new_row])], ignore_index=True)
        
        # Add additional rows for extra images
        for i, image in enumerate(images[1:], 2):
            image_row = {col: '' for col in shopify_columns}
            image_row['Handle'] = handle
            image_row['Image Src'] = image
            image_row['Image Position'] = str(i)
            image_row['Image Alt Text'] = row['Name']
            
            shopify_df = pd.concat([shopify_df, pd.DataFrame([image_row])], ignore_index=True)
    
    # Save to output file
    shopify_df.to_csv(output_file, index=False)
    print(f"Transformation complete! Output saved to: {output_file}")
    
    return shopify_df

print("WooCommerce to Shopify transformation function loaded!")

WooCommerce to Shopify transformation function loaded!


In [8]:
# Run the transformation
from datetime import datetime

input_file = 'inputs/wc_export_1.csv'
current_date = datetime.now().strftime('%Y-%m-%d')
output_file = f'outputs/shopify_transformed_{current_date}.csv'

# Execute the transformation
result_df = transform_wc_to_shopify(input_file, output_file)

print(f"\nProcessed {len(result_df)} rows")
print("\nFirst few products:")
print(result_df[['Handle', 'Title', 'Variant SKU', 'Variant Price', 'Tags']].head())

Transformation complete! Output saved to: outputs/shopify_transformed_2025-08-07.csv

Processed 6 rows

First few products:
                                              Handle  \
0  2024-panini-minecraft-trading-cards-72ct-mega-box   
1  2024-panini-minecraft-trading-cards-72ct-mega-box   
2  2024-panini-minecraft-trading-cards-72ct-mega-box   
3  2024-panini-minecraft-trading-cards-72ct-mega-box   
4  2024-panini-minecraft-trading-cards-72ct-mega-box   

                                               Title              Variant SKU  \
0  2024 Panini Minecraft Trading Cards. 72ct. Meg...  24PnMncrftENT MGB 17127   
1                                                                               
2                                                                               
3                                                                               
4                                                                               

  Variant Price                                     

/var/folders/sw/2fzd_kbd1770h__4kjvs0d300000gn/T/ipykernel_56612/3779934140.py:116: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  shopify_df = pd.concat([shopify_df, pd.DataFrame([new_row])], ignore_index=True)
